In [1]:
from os import listdir
import jsonlines
import itertools
import nltk
from nltk.corpus import stopwords
import pymorphy2 as pm
import networkx as nx
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from collections import Counter
from tqdm import trange
from tqdm import tqdm
from math import log
from random import shuffle
from sys import exit
import numpy as np
import json
import codecs
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.neighbors import NearestCentroid
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
#nltk.download()
tknzr = nltk.TweetTokenizer()

In [2]:
path = listdir('c')
path = sorted(path)[:9]
print(path)

data = []
for p in path:
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            data.append(entry)

['AA', 'AB', 'AD', 'AE', 'AF', 'AI', 'AJ', 'AK', 'AQ']


In [3]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())
    
category_article = {}
with open("sources/cat_article.json", mode="r") as input:
    category_article = json.loads(input.read())
    
texts = {item['id']: item['text'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')}

In [4]:
ids = sorted([id for id in texts.keys()])

### Подготваливаем заголовки

In [5]:
sw_ru = nltk.corpus.stopwords.words('russian')
morph = pm.MorphAnalyzer()
id_title = {}

stems = []
unsup_symb = set('.,\(\):«»?!')
for i in trange(len(data)):
    id = data[i]['id']
    if id in ids:
        tokens = tknzr.tokenize(text=data[i]['title'])
        tokens = [t.lower() for t in tokens if t not in unsup_symb]
        tokens = [morph.parse(t)[0].normal_form for t in tokens]
        tokens = [t for t in tokens if t not in sw_ru]
        if len(tokens) > 0:
            id_title[id] = ' '.join(tokens)
            stems.extend(tokens)
    
stems = set(stems)
print("Done")

100%|█████████████████████████████████████████████████████████████████████████| 116584/116584 [02:49<00:00, 688.48it/s]


Done


In [6]:
title_id = {}
for id in id_title:
    if title_id.get(id_title[id]) is not None:
        title_id[id_title[id]].append(id)
    else:
        title_id[id_title[id]] = [id]

### Вычисляем словарь категорий

In [8]:
categories_vocab = {}

for id in tqdm(categories_dict):
    for c in categories_dict[id]:
        if c in ok:
            if id_title.get(id) is not None:
                if categories_vocab.get(c) is None:
                    categories_vocab[c] = id_title[id]
                else:
                    categories_vocab[c] += ' ' + id_title[id]


for c in categories_vocab:
    categories_vocab[c] = set(categories_vocab[c].split())

100%|██████████████████████████████████████████████████████████████████████████| 96794/96794 [00:27<00:00, 3554.66it/s]


In [7]:
'''with open("sources/categories_vocab", "w") as output:
    output.write(json.dumps({c: sorted(vocab) for c, vocab in categories_vocab.items()}))
'''
with open("sources/categories_vocab", "r") as input:
    categories_vocab = json.loads(input.read())
    categories_vocab = {c: set(vocab) for c, vocab in categories_vocab.items()}

In [8]:
ids = [id for id in ids if id in id_title.keys()]

In [9]:
for id in tqdm(ids):
    tokens = texts[id].split()
    tokens = [tok for tok in tokens if tok in stems]
    texts[id] = ' '.join(tokens)
    
data[:] = []

100%|██████████████████████████████████████████████████████████████████████████| 96769/96769 [00:11<00:00, 8583.74it/s]


### Начинаем анализировать документ

In [10]:
kf = KFold(n_splits=20, shuffle=True, random_state=27)

train_index, test_index = 0, 0
for item in kf.split(ids):
    train_index, test_index = item[0], item[1]
    break

print("TRAIN:", train_index, "TEST:", test_index)
print(len(train_index), len(test_index))

TRAIN: [    0     1     2 ... 96766 96767 96768] TEST: [   56    71    93 ... 96716 96729 96764]
91930 4839


1. Взвешиваем слова в документe по формуле $ R_w = ft_w \log{\frac{N}{cf_w}} $.
2. Взвешиваем заголовки по формуле $R_t = \sum_{w \rightarrow t} R_w \frac{1}{t_w} \frac{1}{a_t} \frac{S_t}{L_t}$.
3. Взвешиваем статьи $R_a = \max_{t \rightarrow a} R_t$ (почти так).
4. Взвештваем категории $R_c = \frac{v_c}{d_c} \sum_{a \rightarrow c}R_a$ и получам ответ.
5. Модификация: обновляем веса категорий по правилу $R_c' = R_c \frac{\sum_{w \in B_c}d_w}{|B_c|}$, $d'_w = \frac{d_w}{2}$ для $d_w \in B_c$ в порядке убывания весов категорий.

In [11]:
topn = 20


results = {}
for i in tqdm(test_index):
    document = texts[ids[i]]
    #Шаг 1
    words = re.split(r' ', document)
    words_set = set(words)
    d_w = {w: 1 for w in words_set}
    # Вычисляем tf_w
    R_w = dict(Counter(words))
    N = len(categories_dict)
    # Вычисляем log(...)
    for w in words_set:
        cf_w = 0
        for c in categories_vocab:
            if w in categories_vocab[c]:
                cf_w += 1
        if cf_w > 0:
            R_w[w] = R_w[w] * log(N / cf_w)
        else:
            R_w[w] = 0
    
    # Шаг 2
    R_t = dict()
    supp_w_t = {}
    title_vocab = dict(Counter(' '.join(title_id.keys()).split()))
    for title in title_id.keys():
        title_words = title.split()
        supp_words = []
        flag = 1
        for w in title_words:
            if w not in words_set:
                flag = flag - 1
            else:
                supp_words.append(w)
        if flag < 0:
            continue
        sub_sum = 0
        # Вынесем 1 / a_t * S_t / L_t как mutual_mul
        mutual_mul = len(supp_words) / (len(title_id[title]) * len(title_words))
        for w in supp_words:
            sub_sum += R_w[w] / title_vocab[w]
        
        R_t[title] = mutual_mul * sub_sum
        supp_w_t[title] = supp_words
    
    # Шаг 3
    #R_a = {d['id']: R_t.get(d['title']) for d in data if R_t.get(d['title']) is not None and R_t.get(d['title']) > 0.0}
    #supp_w_a = {d['id']: supp_w_t.get(d['title']) for d in data if supp_w_t.get(d['title']) is not None 
    #            and  R_t.get(d['title']) > 0.0}
    R_a = {id: R_t.get(title) for (id, title) in id_title.items() if R_t.get(title) is not None and R_t.get(title) > 0.0}
    supp_w_a = {id: supp_w_t.get(title) for (id, title) in id_title.items() if supp_w_t.get(title) is not None 
                and R_t.get(title) > 0.0}
    
    # Шаг 4
    R_c = {}
    for c in ok:
        v_c = 0
        d_c = len(categories_vocab[c])
        r_c = 0
        for id in category_article[c]:
            if R_a.get(id) is not None:
                r_c += R_a.get(id)
                v_c += len(supp_w_a.get(id))
        R_c[c] = r_c * v_c / d_c
    
    # Шаг 5    
    R__c = sorted([k for k in R_c.items() if k[1] > 0], key=lambda t: t[1], reverse=True)[:topn]
    R_c = {}
    for r_c in R__c:
        B_c = set(' '.join([' '.join(supp_w_a.get(id)) for id in category_article[r_c[0]] 
                            if supp_w_a.get(id) is not None]).split())
        sub_sum = 0
        for b in B_c:
            sub_sum += d_w[b]
            d_w[b] = d_w[b] / 2
        
        R_c[r_c[0]] = r_c[1] * sub_sum / len(B_c)
    
    # Влом исправлять
    results[ids[i]] = sorted([k for k in R_c.items() if k[1] > 0], key=lambda t: t[1], reverse=True)[:topn]

100%|████████████████████████████████████████████████████████████████████████████| 4839/4839 [1:05:17<00:00,  1.24it/s]


### Получаем центоиды классов из centroids.json + тренируем бинарный LogReg 

In [11]:
vectorizer = TfidfVectorizer(min_df=5, max_df=1000)
text_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])
svd = TruncatedSVD(n_components=500)
X_svd = svd.fit_transform(text_tfidf)

In [12]:
class_centroids = {}
with open("sources/centroids.json", "r") as input:
    class_centroids = json.loads(input.read())

In [17]:
length = int(len(train_index) / 5)
length

18386

In [49]:
cent_categories = sorted(class_centroids.keys())
X_train = []
y_train = []
for index in tqdm(train_index[:length]):
    cats = categories_dict[ids[index]]
    curr_x = X_svd[index]
    for cat in cats:
        if class_centroids.get(cat) is not None:  
            X_train.append(np.concatenate((curr_x, class_centroids[cat]), axis=0))
            y_train.append(1)
            
    for i in range(20):
        zero_cat = random.choice(list(class_centroids))
        if zero_cat not in cats:
            X_train.append(np.concatenate((curr_x, class_centroids[zero_cat]), axis=0))
            y_train.append(0)


  5%|███▋                                                                         | 890/18386 [00:04<01:29, 195.75it/s]Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\Maxim\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Maxim\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\Maxim\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|███████████████████████████████████████████████████████████████████████████| 18386/18386 [01:28<00:00, 208.40it/s]


In [51]:
X_train = np.array(X_train)

In [54]:
clf = LogisticRegression(C=1.0)
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Теперь отсекаем нехорошие варианты в results

In [78]:
X_test = []
skipped = []
for index in test_index[:length]:
    result = ids[index]
    test_svd = X_svd[index]
    suggested = np.array([res[0] for res in results[result]])
    print(suggested)
    X_local = []
    local_skipped = []
    prediction = []
    for i, sugg in enumerate(suggested):
        if class_centroids.get(sugg) is not None:
            X_local.append(np.concatenate((test_svd, class_centroids[sugg])))
        else:
            local_skipped.append((i, sugg))
    if len(X_local) > 0:
        prediction = clf.predict(X_local)
    for item in local_skipped:
        prediction.insert(item[0], 1)
    print(prediction)
    print('\n\n')

['Умершие в 1929 году' 'Социология' 'Познание' 'Родившиеся в 1864 году'
 'Социальные системы' 'Психология' 'Философские теории' 'Социологи США'
 'Социологи по алфавиту' 'Образ' 'Биология' 'Психолингвистика'
 'Социальная психология' 'Понятия эпистемологии' 'Психические процессы'
 'Теория групп' 'Сознание' 'Общество' 'Социальная философия'
 'Социальные группы']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Григорианский календарь' 'Народы Алжира'
 'Памятники под угрозой уничтожения' 'Имена арабского происхождения'
 'Списки городов по странам' 'Мечети Индии' 'Фильмы на английском языке'
 'Берберы' 'Имена по культуре' 'Методы обнаружения вредоносного кода'
 'Транссахарская торговля' 'Легендариум Толкина' 'Фильмы-драмы США'
 'Колониальная Африка' 'Нигеро-конголезские языки' 'Программа «Аполлон»'
 'Консонантные письменности' 'Кинокомедии США' 'Кочевники'
 'Западная Сахара']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Железнодорожные станции и платформы Москвы' 'Железные дороги СССР'
 'Го

 'Альбомы русского рока' 'Альбомы группы «ДДТ»']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Деятели Французской революции, казнённые на гильотине' 'История Парижа'
 'Григорианский календарь'
 'Люди Великой французской революции по алфавиту' 'Судебная система'
 'Революции' 'Революционеры Франции' 'Фильмы о женской дружбе'
 'Фильмы Великобритании 1992 года' 'Вымышленные государственные детективы'
 'Социализм' 'Должности Российской империи' 'Фильмы о якудза'
 'Учёные Эпохи Возрождения' 'Галло-римские святые' 'Умершие в 1884 году'
 'Фильмы-триллеры Японии' 'История России (1917—1991)'
 'История социализма' 'Парижская коммуна']
[0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]



['Ораторианцы' 'Литература по языкам' 'Искусство' 'Литература Франции'
 'Эссеисты Франции' 'Китайские династии' 'Книги Ветхого Завета'
 'Жанры журналистики' 'Твёрдые формы' 'Методы исследования'
 'Литературные журналы' 'Авторское право'
 'Шестнадцать варварских государств' 'Образ' 'Литература Древней Греции'
 'Альбомы 

 'Короли Чехии']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Метан' 'Заморские территории Великобритании' 'Фильмы ужасов 1978 года'
 'Фильмы на английском языке' 'Городские легенды' 'Квантовые явления'
 'Паранормальные явления' 'Супрамолекулярная химия'
 'Англоязычные страны и территории' 'Понятия эпистемологии'
 'Теоретические основы баз данных' 'Чёрные дыры'
 'Радиостанции на русском языке' 'Фильмы-триллеры Италии'
 'Классификация самолётов' 'Газы' 'Кинокомедии США' 'Топливо'
 'Геометрия треугольника' 'Релятивистские и гравитационные явления']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Родившиеся в 1833 году' 'Губернии РСФСР' 'Григорианский календарь'
 'Умершие в 1881 году' 'Техническая документация'
 'Учёные степени и звания' 'Медицинская диагностика' 'Фильмы о Шаолине'
 'Телепередачи Первого канала' 'Имена по культуре' 'Математическая логика'
 'Авторы учебников' 'Защитные механизмы' 'Первый канал' 'Буквы кириллицы'
 'Философские термины' 'Гуманистическая психология' 'Брюс Ли

 'Режущие инструменты' 'Сталь']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Энергия' 'Гидравлика' 'Интегральное исчисление' 'Физические законы'
 'Масса' 'Гидродинамика' 'Единицы измерения объёма' 'Законы сохранения'
 'Физические величины' 'Интегралы' 'Физические законы и уравнения'
 'Статистическая физика' 'Химические законы и уравнения' 'Скорость'
 'Физическая химия' 'Механика' 'Химическая термодинамика'
 'Планковские единицы' 'Физические константы' 'Динамика']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Катастеризмы' 'Новые созвездия' 'Часы' 'Широта' 'Миграция населения'
 'Зодиакальные созвездия' 'Члены Французской академии наук'
 'Физические эксперименты' 'Лексикология' 'Город'
 'Члены Лондонского королевского общества' 'Геодезия'
 'Отменённые созвездия'
 'Западное Средиземноморье в древнегреческой мифологии'
 'Районы Рейнланд-Пфальца' 'Основатели Королевского общества'
 'История физики' 'Объекты Флемстида'
 'Почётные члены Петербургской академии наук' 'Фотоматериалы']
[0 0 0 

 'Русские в Латвии']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Железнодорожный транспорт' 'Подвижной состав железных дорог'
 'Проезды Москвы' 'Транспорт' 'Грузовые вагоны' 'Электропоезда РВЗ'
 'Пассажирские вагоны' 'Трансформаторы' 'Железнодорожная инфраструктура'
 'Общество потребления' 'Конструкция летательных аппаратов'
 'Электропоезда СССР' 'Альбомы группы «ДДТ»' 'Железные дороги СССР'
 'Путевое хозяйство' 'Марксистские понятия' 'Локомотивы' 'Электричество'
 'Подвижной состав Московского метрополитена' 'Железные дороги России']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Нижние палаты парламентов по странам' 'Избирательные округа Науру'
 'Парламент' 'Левоцентристские партии' 'Григорианский календарь'
 'Санкт-Галлен (кантон)' 'Списки политических партий' 'Выборы'
 'Парламенты по странам' 'Третий сезон «Остаться в живых»'
 'Геологические периоды' 'Дни недели' 'Социал-демократические партии'
 'Либерально-консервативные партии' 'Политика' 'Стандартизация'
 'Политические партии 

...

 'Обладатели приза Луи Деллюка' 'Фильмы о Вашингтоне']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Фильмы на английском языке' 'Фильмы ужасов о животных' 'Кинокомедии США'
 'Семейства паукообразных' 'Фильмы ужасов США' 'Списки городов по странам'
 'Фильмы СССР 1958 года' 'Доллар' 'Типы животных' 'Кинокомедии 1990 года'
 'Фильмы Hollywood Pictures' 'Фильмы-драмы США'
 'Фильмы Amblin Entertainment' 'Чёрно-белые фильмы СССР'
 'Фильмы-драмы СССР' 'Фильмы киностудии имени М. Горького'
 'Североамериканские валюты' 'Перевод' 'Фильмы-триллеры США'
 'Фильмы ужасов 1990 года']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Фильмы США 1967 года' 'Фильмы-драмы США' 'Фильмы Марка Донского'
 'Фильмы на английском языке' 'Молодёжные фильмы' 'Кинокомедии США'
 'Классификация самолётов' 'Фильмы СССР 1947 года' 'Фильмы ужасов США'
 'Фильмы-драмы Канады' 'Системы счисления' 'Конкурсы красоты'
 'Чёрно-белые фильмы СССР' 'Фильмы-триллеры США'
 'Лидеры советского кинопроката' 'Фильмы Imagine Entertainmen

 'Литературные жанры' 'Революционеры Китая']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Годы в театре' 'Фильмы на английском языке' 'Фильмы-драмы США'
 'Драматургия' 'Категории эстетики' 'Социология культуры'
 'Композиторы Японии' 'Кинопрофессии' 'Детские игры'
 'Лауреаты почётной премии «Оскар»' 'Фильмы-триллеры США'
 'Фильмы о самураях' 'Фильмы — лауреаты кинофестиваля «Окно в Европу»'
 'Лауреаты премии Киото' 'Образ' 'Фильмы-драмы Японии'
 'Фильмы-драмы Франции' 'Фантастические фильмы 1985 года'
 'Фильмы-боевики США' 'Чёрно-белые фильмы Японии']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Фильмы-драмы Венгрии' 'Фильмы-драмы Австрии'
 'Фильмы на тематику мужской гомосексуальности' 'Фильмы-драмы ФРГ'
 'Фильмы на английском языке' 'Фильмы-биографии Германии' 'Книги'
 'Бюрократия' 'Фильмы Жана-Люка Годара' 'Наказания'
 'Приказы Русского государства' 'Фильмы Великобритании 1993 года'
 'Кинокомедии США' 'Христианские праздники' 'Литература'
 'Фильмы-драмы США' 'Фильмы-мелодрамы США

 'Фильмы ужасов 1977 года' 'Фильмы о Вашингтоне']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]



['Слова' 'Мировоззрение' 'Фильмы ужасов Канады'
 'Юридические должности и профессии' 'Краут-рок-музыканты' 'Антропология'
 'Комедийные фильмы о полицейских' 'Фильмы на английском языке'
 'Фильмы о кошках' 'Фильмы Франции 1974 года' 'Путевые машины'
 'Фильмы ужасов 1977 года' 'Танатология'
 'Уничижительные обозначения людей' 'Can'
 'Романтические кинокомедии Великобритании' 'Терминальные состояния'
 'Экспериментальные музыканты' 'Фильмы-триллеры США' 'Литературоведение']
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]



['Фильмы об инопланетянах' 'Астрономические обсерватории'
 'Планеты Солнечной системы' 'Фантастические комедии США' 'Энергия'
 'Кинокомедии США' 'Научно-фантастические фильмы США' 'Погода'
 'Военные фильмы СССР' 'Астрономические объекты'
 'Телепередачи Первого канала' 'Фильмы Стивена Спилберга'
 'Фильмы на английском языке' 'Фильмы-мелодрамы СССР'
 'Фильмы США 1977 года' 'Фильмы США

 'Кинокомедии 1987 года' 'Чёрно-белые фильмы СССР']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Фильмы на английском языке' 'Фильмы-триллеры США' 'Фильмы-драмы США'
 'Фильмы-драмы Франции' 'Опиаты' 'Кинокомедии США'
 'Чёрно-белые фильмы СССР' 'Объекты, названные в честь Максима Горького'
 'Фильмы студии «Мосфильм»' 'Фильмы киностудии имени М. Горького'
 'Фильмы СССР 1962 года' 'Кинокомпании России' 'Детективные фильмы США'
 'Фильмы-боевики США' 'Фильмы Франции 1980 года' 'Фильмы-драмы СССР'
 'Кинокомпании СССР' 'Лидеры советского кинопроката'
 'Национальный реестр фильмов' 'Фильмы-мелодрамы США']
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]



['Эротические фильмы Франции' 'Фильмы на английском языке'
 'Фильмы-драмы США' 'Кинокомедии США' 'Киноальманахи'
 'Фильмы-драмы Франции' 'Мультфильмы 2002 года'
 'Телепередачи Первого канала' 'Жанры кино' 'Человеческое поведение'
 'Виды и жанры театра' 'Первый канал' 'Фильмы-триллеры США'
 'Фильмы Франции 1974 года' 'Проституция в фильмах'
 'Филь

KeyboardInterrupt: 

### Тут то же самое, что и выше, только выбираем k ближайших центроидов, а не истинные + n рандомных

In [10]:
k_nearest = 5

In [11]:
class_centroids = {}
with open("sources/centroids.json", "r") as input:
    class_centroids = json.loads(input.read())

In [12]:
kn_clust = NearestNeighbors(n_neighbors=k_nearest)

In [13]:
kn_clust.fit([centroid for (cat, centroid) in sorted(class_centroids.items())])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [ ]:
'''vectorizer = TfidfVectorizer(min_df=5, max_df=1000)
text_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])
svd = TruncatedSVD(n_components=500)
X_svd = svd.fit_transform(text_tfidf)'''

X_svd = np.load("sources/svd.npy")

In [83]:
length = int(len(train_index) / 5)
length

18386

In [84]:
cent_categories = sorted(class_centroids.keys())
X_train = []
y_train = []
for index in tqdm(train_index):
    cats = categories_dict[ids[index]]
    curr_x = X_svd[index]
    curr_x = curr_x.reshape(1,len(curr_x))
    neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
    neighbors = [ok[c] for c in neighbors[0]]
    common = set(neighbors).intersection(set(cats))
    for neigh in neighbors:
        X_train.append(np.concatenate((curr_x[0], class_centroids[neigh]), axis=0))
        y_train.append(1 if neigh in common else 0)


  0%|                                                                                | 7/91930 [00:00<25:53, 59.16it/s]Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\Users\Maxim\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Maxim\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\Maxim\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|████████████████████████████████████████████████████████████████████████████| 91930/91930 [26:40<00:00, 57.45it/s]


In [85]:
clf = LogisticRegression(C=1.0)
clf.fit(np.array(X_train), np.array(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
X_test = []
skipped = []
for index in test_index[:50]:
    result = ids[index]
    test_svd = X_svd[index]
    suggested = np.array([res[0] for res in results[result]])
    print(suggested)
    X_local = []
    local_skipped = []
    prediction = []
    for i, sugg in enumerate(suggested):
        if class_centroids.get(sugg) is not None:
            X_local.append(np.concatenate((test_svd, class_centroids[sugg])))
        else:
            local_skipped.append((i, sugg))
    if len(X_local) > 0:
        prediction = clf.predict(X_local)
    for item in local_skipped:
        prediction.insert(item[0], 1)
    print(prediction)
    print('\n\n')

['Умершие в 1929 году' 'Социология' 'Познание' 'Родившиеся в 1864 году'
 'Социальные системы' 'Психология' 'Философские теории' 'Социологи США'
 'Социологи по алфавиту' 'Образ' 'Биология' 'Психолингвистика'
 'Социальная психология' 'Понятия эпистемологии' 'Психические процессы'
 'Теория групп' 'Сознание' 'Общество' 'Социальная философия'
 'Социальные группы']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Григорианский календарь' 'Народы Алжира'
 'Памятники под угрозой уничтожения' 'Имена арабского происхождения'
 'Списки городов по странам' 'Мечети Индии' 'Фильмы на английском языке'
 'Берберы' 'Имена по культуре' 'Методы обнаружения вредоносного кода'
 'Транссахарская торговля' 'Легендариум Толкина' 'Фильмы-драмы США'
 'Колониальная Африка' 'Нигеро-конголезские языки' 'Программа «Аполлон»'
 'Консонантные письменности' 'Кинокомедии США' 'Кочевники'
 'Западная Сахара']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Железнодорожные станции и платформы Москвы' 'Железные дороги СССР'
 'Го

 'Альбомы русского рока' 'Альбомы группы «ДДТ»']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Деятели Французской революции, казнённые на гильотине' 'История Парижа'
 'Григорианский календарь'
 'Люди Великой французской революции по алфавиту' 'Судебная система'
 'Революции' 'Революционеры Франции' 'Фильмы о женской дружбе'
 'Фильмы Великобритании 1992 года' 'Вымышленные государственные детективы'
 'Социализм' 'Должности Российской империи' 'Фильмы о якудза'
 'Учёные Эпохи Возрождения' 'Галло-римские святые' 'Умершие в 1884 году'
 'Фильмы-триллеры Японии' 'История России (1917—1991)'
 'История социализма' 'Парижская коммуна']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Ораторианцы' 'Литература по языкам' 'Искусство' 'Литература Франции'
 'Эссеисты Франции' 'Китайские династии' 'Книги Ветхого Завета'
 'Жанры журналистики' 'Твёрдые формы' 'Методы исследования'
 'Литературные журналы' 'Авторское право'
 'Шестнадцать варварских государств' 'Образ' 'Литература Древней Греции'
 'Альбомы 

 'Короли Чехии']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Метан' 'Заморские территории Великобритании' 'Фильмы ужасов 1978 года'
 'Фильмы на английском языке' 'Городские легенды' 'Квантовые явления'
 'Паранормальные явления' 'Супрамолекулярная химия'
 'Англоязычные страны и территории' 'Понятия эпистемологии'
 'Теоретические основы баз данных' 'Чёрные дыры'
 'Радиостанции на русском языке' 'Фильмы-триллеры Италии'
 'Классификация самолётов' 'Газы' 'Кинокомедии США' 'Топливо'
 'Геометрия треугольника' 'Релятивистские и гравитационные явления']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Родившиеся в 1833 году' 'Губернии РСФСР' 'Григорианский календарь'
 'Умершие в 1881 году' 'Техническая документация'
 'Учёные степени и звания' 'Медицинская диагностика' 'Фильмы о Шаолине'
 'Телепередачи Первого канала' 'Имена по культуре' 'Математическая логика'
 'Авторы учебников' 'Защитные механизмы' 'Первый канал' 'Буквы кириллицы'
 'Философские термины' 'Гуманистическая психология' 'Брюс Ли

 'Режущие инструменты' 'Сталь']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



['Энергия' 'Гидравлика' 'Интегральное исчисление' 'Физические законы'
 'Масса' 'Гидродинамика' 'Единицы измерения объёма' 'Законы сохранения'
 'Физические величины' 'Интегралы' 'Физические законы и уравнения'
 'Статистическая физика' 'Химические законы и уравнения' 'Скорость'
 'Физическая химия' 'Механика' 'Химическая термодинамика'
 'Планковские единицы' 'Физические константы' 'Динамика']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]





### Тут кроссваливация бинарной лог регрессии

In [ ]:
n_splits = 5
k_nearest = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=27)
clf = LogisticRegression(C=0.1)
X_train = []
y_train = []

train_index, test_index = 0, 0
for item in kf.split(ids):
    
    score = 0
    train_index, test_index = item[0], item[1]
    
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(train_index), len(test_index))
    
    X_train[:] = []
    y_train[:] = []
    for index in tqdm(train_index):
        cats = categories_dict[ids[index]]
        curr_x = X_svd[index]
        curr_x = curr_x.reshape(1,len(curr_x))
        neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
        neighbors = [ok[c] for c in neighbors[0]]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            X_train.append(np.concatenate((curr_x[0], class_centroids[neigh]), axis=0))
            y_train.append(1 if neigh in common else 0)
    
    clf.fit(np.array(X_train), np.array(y_train))
    
    for index in test_index:
        result = ids[index]
        test_svd = X_svd[index]
        y_true = np.array(categories_dict[result])
        X_local = []
        local_skipped = []
        prediction = []
        for i, y in enumerate(y_true):
            if class_centroids.get(y) is not None:
                X_local.append(np.concatenate((test_svd, class_centroids[y])))
            else:
                local_skipped.append((i, y))
        if len(X_local) > 0:
            prediction = clf.predict(X_local)
        for item in local_skipped:
            prediction.insert(item[0], 1)
            
        score += sum(prediction) / len(y_true)
    
    print(score)    

TRAIN: [    0     1     2 ... 96766 96767 96768] TEST: [    4     7    27 ... 96754 96755 96764]
77415 19354


 98%|██████████████████████████████████████████████████████████████████████████▎ | 75704/77415 [21:27<00:29, 58.78it/s]

c | k_nearest | score
:-:|:------:|:---:
1.0 | 5 | 0.06